## Question 1

You want to retire in 30 years with $4,000,000 in your retirement account.  If your investments yield an annual return of 8% and you currently have no savings, how much should you save each year to reach your retirement goal? 

In [3]:
# Method 1

import numpy_financial as npf
answer1 = -npf.pmt(rate=0.08, nper=30, pv=0, fv=4000000)

# Method 2

import numpy as np
fv_factor = np.sum(
    1.08 ** np.arange(29, -1, -1)
)
answer2 = 4000000 / fv_factor

print(f"answer by method 1 is ${answer1:,.2f}")
print(f"answer by method 2 is ${answer2:,.2f}")

answer by method 1 is $35,309.73
answer by method 2 is $35,309.73


## Question 2

Suppose in #1 that your returns are normally distributed with a mean of 8% and a standard deviation of 15%.  Suppose you save the amount you calculated in #1.  Run a simulation to determine the probability that you will meet your retirement goal.

Our first savings will be at the end of year 1 and will earn returns for 29 years, our next for 28 years, etc.  The last savings will be at the end of year 30 and will not earn any returns.  So, we need to generate 29 random annual returns in each simulation.  First we generate the returns.

In [36]:
num_sims = 10000

np.random.seed(0)
rets = np.random.normal(
    loc=0.08, scale=0.15, size=(29, num_sims)
)

The first savings will earn the compound return $(1+r_2) \cdots (1+r_30)$ where $r_t$ is the return in year $t$.  The second will earn $(1+r_3) \ldots (1+r_30)$, etc.  Multiplying the savings by the compound return and then adding over the years of savings is the same as adding the compound returns over the years of savings and then adding.  So, we need to add up the compound returns.  First, we will do it in a nested loop and then in a slightly quicker way.

In [37]:
sum_compound_returns_method1 = []
for sim in range(num_sims):
    sum = 0
    for year in range(29):
        sum += np.prod((1+rets[year:, sim]))
    sum_compound_returns_method1.append(sum)
    
sum_compound_returns_method1 = np.array(sum_compound_returns_method1)



In [38]:
sum_compound_returns_method2 = np.sum(
    np.flip(
        np.cumprod(
            np.flip(1+rets, axis=0), 
            axis=0
        ), 
        axis=0
    ), 
    axis=0
)

We can check that the two methods give the same result.

In [39]:
np.sum((sum_compound_returns_method1 - sum_compound_returns_method2)**2)

9.938161191839312e-24

The question is in what fraction of the simulations we have

$$\text{savings} \times \text{sum of compound returns} >= \text{\$4,000,000}$$

In [40]:
savings = 35309.73
sum_compound_returns = sum_compound_returns_method1

success = savings * sum_compound_returns >= 4000000

print(f"probability of achieving target is {np.sum(success)/num_sims:.2%}")

probability of achieving target is 38.14%


### Question 3

Repeat #2 assuming the standard deviation is only 10%.  What is the probability of meeting your retirement goal?

In [41]:
num_sims = 10000

np.random.seed(0)
rets = np.random.normal(
    loc=0.08, scale=0.10, size=(29, num_sims)
)
sum_compound_returns = np.sum(
    np.flip(
        np.cumprod(
            np.flip(1+rets, axis=0), 
            axis=0
        ), 
        axis=0
    ), 
    axis=0
)

savings = 35309.73
success = savings * sum_compound_returns >= 4000000

print(f"probability of achieving target is {np.sum(success)/num_sims:.2%}")

probability of achieving target is 41.44%


### Question 4

Use the following code to calculate daily SPY returns.  Calculate the skewness and kurtosis of the returns using the formulas presented in class.

    import yfinance as yf
    price = yf.download("spy", start="1990-01-01")["Adj Close"]
    ret = price.pct_change()

In [45]:
import yfinance as yf
price = yf.download("spy", start="1990-01-01")["Adj Close"]
ret = price.pct_change()

mean = ret.mean()
variance = np.mean((ret-mean)**2)
third = np.mean((ret-mean)**3)
fourth = np.mean((ret-mean)**4)
stdev = np.sqrt(variance)
skewness = third / stdev**3
kurtosis = fourth / stdev**4
print(f"skewness is {skewness:.3f}")
print(f"kurtosis is {kurtosis:.3f}")

[*********************100%***********************]  1 of 1 completed
skewness is -0.056
kurtosis is 14.308


### Question 5

Use the following code to calculate annual SPY returns.  Calculate the skewness and kurtosis of the returns using the formulas presented in class.

    import yfinance as yf
    price = yf.download("spy", start="1990-01-01")["Adj Close"]
    price = price.resample("Y").last()
    ret = price.pct_change()[:-1]

In [63]:
price = price.resample("Y").last()
ret = price.pct_change()[1:-1]

mean = ret.mean()
variance = np.mean((ret-mean)**2)
third = np.mean((ret-mean)**3)
fourth = np.mean((ret-mean)**4)
stdev = np.sqrt(variance)
skewness = third / stdev**3
kurtosis = fourth / stdev**4
print(f"skewness is {skewness:.3f}")
print(f"kurtosis is {kurtosis:.3f}")

skewness is -0.748
kurtosis is 2.990


### Question 6

Download the CPI from FRED and calculate annual inflation rates as the percent change in the end-of-year CPI from one year to the next.  Using the annual (nominal) returns from #5, calculate the real rate of return of SPY each year.


In [59]:
from pandas_datareader import DataReader as pdr

cpi = pdr('CPIAUCSL', 'fred', start="1993-01-01", end="2022-12-31")
cpi = cpi.resample('Y').last()

inflation = cpi.pct_change().dropna()
inflation = inflation.iloc[:, 0]
inflation.head()

DATE
1994-12-31    0.025974
1995-12-31    0.025316
1996-12-31    0.033788
1997-12-31    0.016970
1998-12-31    0.016069
Freq: A-DEC, Name: CPIAUCSL, dtype: float64

The date formats are not the same, so the indexes don't match.  We could change one of the indexes, but the simplest thing is to convert both series to numpy arrays and then convert back to pandas at the end.

In [64]:
import pandas as pd 
real_ret = (1+ret.to_numpy()) / (1+inflation.to_numpy()) - 1
real_ret = pd.Series(real_ret, index=inflation.index)
real_ret.head()

DATE
1994-12-31   -0.021444
1995-12-31    0.346403
1996-12-31    0.184938
1997-12-31    0.312479
1998-12-31    0.266572
Freq: A-DEC, dtype: float64